<a href="https://colab.research.google.com/github/a-forty-two/GAIP_12_08_21/blob/main/03_CV_deep_dive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Augmentor

In [2]:
import Augmentor
pipes = Augmentor.Pipeline('pics')

Initialised with 3 image(s) found.
Output directory set to pics/output.

In [3]:
pipes.flip_left_right(0.5)
pipes.black_and_white(0.1)
pipes.random_distortion(1, 10, 10, 1)
pipes.rotate(0.5, 25, 25)
pipes.skew(0.25, 0.5)

pipes.sample(100)
# Data transformation-> Apache Beam

Processing <PIL.Image.Image image mode=1 size=2500x1876 at 0x7F46B901A090>: 100%|██████████| 100/100 [00:30<00:00,  3.29 Samples/s]


In [11]:
!ls pics/output

!zip -r my.zip pics


pics_original_001-img.jpeg_02eb5750-61c4-4f84-892b-4764dd067808.jpeg
pics_original_001-img.jpeg_17fe4730-c232-4985-b910-7b9df43c0cbf.jpeg
pics_original_001-img.jpeg_1c4923b1-fb02-4c73-a906-d835b2caad99.jpeg
pics_original_001-img.jpeg_1e5bc843-948e-4eae-8a4e-4359e7b51c6d.jpeg
pics_original_001-img.jpeg_24c081c1-b6ed-4b2f-aea0-88c8b365f6d4.jpeg
pics_original_001-img.jpeg_2d01730d-49bc-4fab-8c26-05abf6f35be3.jpeg
pics_original_001-img.jpeg_2e11d40d-8530-4871-9075-ef09e23be115.jpeg
pics_original_001-img.jpeg_310fc1da-83c8-4f74-9325-de1af282712b.jpeg
pics_original_001-img.jpeg_4ea1c1fa-276f-4db2-8e1a-c07967139132.jpeg
pics_original_001-img.jpeg_7bbe5e61-e866-4e63-9447-c228ac753edd.jpeg
pics_original_001-img.jpeg_81bb8528-e333-4549-a65d-ef6f7f4f1075.jpeg
pics_original_001-img.jpeg_842d7e07-0617-4fb6-aa26-ebeb663b5464.jpeg
pics_original_001-img.jpeg_868e50b9-1e04-4bfb-9bf7-aaf62e4c22d6.jpeg
pics_original_001-img.jpeg_8ed7f0f6-d92d-454f-aaa9-114e0fc5b2d7.jpeg
pics_original_001-img.jpeg_9b2131a

In [13]:
import requests
from requests import exceptions
import cv2
import os

In [17]:
HP_initial_offset = 0
HP_group_size = 50
currentOffset = HP_initial_offset
accesskey = '40582630fa6446b6b37f53a84e4edce0'
search_url = "https://api.bing.microsoft.com/v7.0/images/search"
# url for web search: "https://api.bing.microsoft.com/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": accesskey}
myexceptions = {IOError, FileNotFoundError, exceptions.RequestException, exceptions.HTTPError, 
                exceptions.ConnectionError, exceptions.Timeout}
search_term = "mantis shrimp"


In [18]:
params = {"q": search_term, "offset":HP_initial_offset, "count":HP_group_size,"textDecorations": True, "textFormat": "HTML"}
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()

In [19]:
search_results = response.json()

In [20]:
search_results

{'_type': 'Images',
 'currentOffset': 0,
 'instrumentation': {'_type': 'ResponseInstrumentation'},
 'nextOffset': 63,
 'pivotSuggestions': [{'pivot': 'mantis shrimp',
   'suggestions': [{'displayText': 'Odontodactylus Scyllarus',
     'searchLink': 'https://api.bing.microsoft.com/api/v7/images/search?q=Odontodactylus+Scyllarus&tq=%7b%22pq%22%3a%22mantis+shrimp%22%2c%22qs%22%3a%5b%7b%22cv%22%3a%22mantis+shrimp%22%2c%22pv%22%3a%22mantis+shrimp%22%2c%22hps%22%3atrue%2c%22iqp%22%3afalse%7d%2c%7b%22cv%22%3a%22Odontodactylus+Scyllarus%22%2c%22pv%22%3a%22%22%2c%22hps%22%3afalse%2c%22iqp%22%3atrue%7d%5d%7d',
     'text': 'Odontodactylus Scyllarus',
     'thumbnail': {'thumbnailUrl': 'https://tse3.mm.bing.net/th?q=Odontodactylus+Scyllarus&pid=Api&mkt=en-US&adlt=moderate&t=1'},
     'webSearchUrl': 'https://www.bing.com/images/search?q=Odontodactylus+Scyllarus&tq=%7b%22pq%22%3a%22mantis+shrimp%22%2c%22qs%22%3a%5b%7b%22cv%22%3a%22mantis+shrimp%22%2c%22pv%22%3a%22mantis+shrimp%22%2c%22hps%22%3atru

In [21]:
total = 0
downloadedCount = min(HP_group_size,search_results['totalEstimatedMatches'] )
os.mkdir(search_term)
currentOffset = 0

In [22]:
for offset in range(0, downloadedCount, HP_group_size): 
  params['offset'] = currentOffset
  response = requests.get(search_url, headers=headers, params=params)
  response.raise_for_status()
  search_results = response.json()
  for res in search_results["value"]:
    try:
      r = requests.get(res['contentUrl'], timeout=30)
      ext = res['encodingFormat']
      fname = search_term + '/'+  str(total).zfill(8) + '.' + ext
      fl = open(fname, 'wb')
      fl.write(r.content)
      fl.close()

      img = cv2.imread(fname)
      if img is None:
        print('bad file detected, deleting the file')
        os.remove(fname) 
        continue
      img1 =cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      
      total = total + 1
      fname = search_term + '/'+  str(total).zfill(8) + '.' + ext
      cv2.imwrite(fname, img1)
      total = total + 1
  
    except Exception as e:
      print(e)
      if type(e) in myexceptions:
        continue
  currentOffset = search_results['totalEstimatedMatches']


bad file detected, deleting the file
bad file detected, deleting the file
bad file detected, deleting the file
